In [1]:
#pip install pyarrow
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
import pickle

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [3]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [4]:
import mlflow
mlflow.set_tracking_uri("sqlite:///prediction.db")
mlflow.set_experiment("nycity-taxi-experiment")

<Experiment: artifact_location='./mlruns/1', creation_time=1664455787663, experiment_id='1', last_update_time=1664455787663, lifecycle_stage='active', name='nycity-taxi-experiment', tags={}>

In [18]:

def read_dataframe(filename):

    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >=0) & (df.duration <=60)]
    numerical = ['trip_distance']
    categorical = ['PULocationID', 'DOLocationID']
    train_dict = df[categorical + numerical].to_dict(orient = 'records')
    df[categorical] = df[categorical].astype(str)

    return df


In [19]:
df_train = read_dataframe('../data/green_tripdata_2021-01.parquet')
df_train.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,duration
0,2,2021-01-01 00:15:56,2021-01-01 00:19:52,N,1.0,43,151,1.0,1.01,5.5,...,0.5,0.00,0.0,NaN,0.3,6.80,2.0,1.0,0.00,3.933333
1,2,2021-01-01 00:25:59,2021-01-01 00:34:44,N,1.0,166,239,1.0,2.53,10.0,...,0.5,2.81,0.0,NaN,0.3,16.86,1.0,1.0,2.75,8.750000
2,2,2021-01-01 00:45:57,2021-01-01 00:51:55,N,1.0,41,42,1.0,1.12,6.0,...,0.5,1.00,0.0,NaN,0.3,8.30,1.0,1.0,0.00,5.966667
3,2,2020-12-31 23:57:51,2021-01-01 00:04:56,N,1.0,168,75,1.0,1.99,8.0,...,0.5,0.00,0.0,NaN,0.3,9.30,2.0,1.0,0.00,7.083333
4,2,2021-01-01 00:16:36,2021-01-01 00:16:40,N,2.0,265,265,3.0,0.00,-52.0,...,-0.5,0.00,0.0,NaN,-0.3,-52.80,3.0,1.0,0.00,0.066667


In [20]:
df_val = read_dataframe('../data/green_tripdata_2021-02.parquet')
df_val.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,duration
0,2,2021-02-01 00:34:03,2021-02-01 00:51:58,N,1.0,130,205,5.0,3.66,14.0,...,0.5,10.00,0.0,NaN,0.3,25.30,1.0,1.0,0.00,17.916667
1,2,2021-02-01 00:04:00,2021-02-01 00:10:30,N,1.0,152,244,1.0,1.10,6.5,...,0.5,0.00,0.0,NaN,0.3,7.80,2.0,1.0,0.00,6.500000
2,2,2021-02-01 00:18:51,2021-02-01 00:34:06,N,1.0,152,48,1.0,4.93,16.5,...,0.5,0.00,0.0,NaN,0.3,20.55,2.0,1.0,2.75,15.250000
3,2,2021-02-01 00:53:27,2021-02-01 01:11:41,N,1.0,152,241,1.0,6.70,21.0,...,0.5,0.00,0.0,NaN,0.3,22.30,2.0,1.0,0.00,18.233333
4,2,2021-02-01 00:57:46,2021-02-01 01:06:44,N,1.0,75,42,1.0,1.89,8.5,...,0.5,2.45,0.0,NaN,0.3,12.25,1.0,1.0,0.00,8.966667


In [21]:
df_train['PU_DU'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DU'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [22]:
categorical = ['PU_DU']#'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_Dict = df_train[categorical + numerical].to_dict(orient = 'records')
X_train = dv.fit_transform(train_Dict)

val_Dict = df_val[categorical + numerical].to_dict(orient = 'records')
X_val = dv.transform(val_Dict)

In [23]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [12]:


lr = LinearRegression().fit(X_train, y_train)
y_pred = lr.predict(X_val)
mean_squared_error(y_val, y_pred, squared = False)

7.808922487065229

In [13]:
with open('model/lin_red.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [13]:
with mlflow.start_run():
    
    mlflow.set_tag('mleng','jayesh')
    mlflow.log_param('train_data','../data/green_tripdata_2021-01.parquet ')
    mlflow.log_param('val_data','../data/green_tripdata_2021-02.parquet')

    alpha = 0.1
    mlflow.log_param('alpha', alpha)
    lr = Lasso(alpha = alpha).fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared = False)
    mlflow.log_metric('rmse',rmse)

    mlflow.log_artifact(local_path='model/lin_red.bin', artifact_path='models_pickle')


In [24]:
train = xgb.DMatrix(X_train, label = y_train)
valid = xgb.DMatrix(X_val, label = y_val)

In [25]:
def objective(params):

    with mlflow.start_run():
        mlflow.set_tag('model','xgboost')
        mlflow.log_params(params)
        booster = xgb.train(params = params,
                dtrain = train,
                num_boost_round = 1000,
                evals =[(valid,'validation')],
                early_stopping_rounds = 50)
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared = False)
        mlflow.log_metric('rmse', rmse)

    return {'loss':rmse,'status':STATUS_OK}

In [26]:
# search_space = {
#     'max_depth': scope.int(hp.quniform('max_depth', 4,100,1)),
#     'learning_rate': hp.loguniform('learning_rate', -3,0),
#     'reg_alpha':hp.loguniform('reg_alpha', -5,-1),
#     'reg_lambda':hp.loguniform('reg_lambda', -6,1),
#     'min_child_weight': hp.loguniform('min_child_weight', -1,3),
#     'objective':'reg:linear',
#     'seed':42
# }
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4,100,1)),
    'seed':42
}
best_result = fmin(
    fn = objective,
    space = search_space,
    algo = tpe.suggest,
    max_evals = 50,
    trials = Trials()
)

[0]	validation-rmse:15.75198                          
[1]	validation-rmse:12.23396                          
[2]	validation-rmse:9.94506                           
[3]	validation-rmse:8.53379                           
[4]	validation-rmse:7.69776                           
[5]	validation-rmse:7.19981                           
[6]	validation-rmse:6.92305                           
[7]	validation-rmse:6.75088                           
[8]	validation-rmse:6.65238                           
[9]	validation-rmse:6.58923                           
[10]	validation-rmse:6.55055                          
[11]	validation-rmse:6.51752                          
[12]	validation-rmse:6.49678                          
[13]	validation-rmse:6.48406                          
[14]	validation-rmse:6.47458                          
[15]	validation-rmse:6.46815                          
[16]	validation-rmse:6.46285                          
[17]	validation-rmse:6.45678                          
[18]	valid

KeyboardInterrupt: 

In [17]:
# best parameter fit:

params = {
    'learning_rate': 0.20472,
    'max_depth': 17,
    'min_child_weight': 1.240261172,
    'objective':'reg:linear',
    'reg_alpha':0.285678967,
    'reg_lambda': 0.004264404814,
    'seed':42
}

mlflow.xgboost.autolog()
booster = xgb.train(params = params,
                dtrain = train,
                num_boost_round = 1000,
                evals =[(valid,'validation')],
                early_stopping_rounds = 50)

2022/10/16 11:37:26 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'a01392119ecc4632913598733f0236f9', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[11:37:26] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:17.44041
[1]	validation-rmse:14.68804
[2]	validation-rmse:12.58747
[3]	validation-rmse:11.00562
[4]	validation-rmse:9.82595
[5]	validation-rmse:8.96148
[6]	validation-rmse:8.33128
[7]	validation-rmse:7.87864
[8]	validation-rmse:7.55235
[9]	validation-rmse:7.31517
[10]	validation-rmse:7.14547
[11]	validation-rmse:7.02208
[12]	validation-rmse:6.93134
[13]	validation-rmse:6.86199
[14]	validation-rmse:6.81195
[15]	validation-rmse:6.77212
[16]	validation-rmse:6.74095
[17]	validation-rmse:6.71596
[18]	validation-rmse:6.69795
[19]	validation-rmse:6.68378
[20]	validation-rmse:6.67428
[21]	validation-rmse:6.66679
[22]	validation-rmse:6.66122
[23]	validation-rmse:6.65665
[24]	validation-rmse:6.65238
[25]	validation-rmse:6.64856
[26]	validation-rmse:6.64467
[27]	validation-rmse:6.64121
[28]	validation-rmse:6.63796
[29]	validation-rmse:6.63532
[30]	validation-r

2022/10/16 11:54:22 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2022/10/16 11:54:25 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/Asus/miniconda3/envs/Mlflow/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."


In [18]:
mlflow.xgboost.autolog(disable=True)

In [19]:
with mlflow.start_run():
    params = {
    'learning_rate': 0.20472,
    'max_depth': 17,
    'min_child_weight': 1.240261172,
    'objective':'reg:linear',
    'reg_alpha':0.285678967,
    'reg_lambda': 0.004264404814,
    'seed':42
    }
    mlflow.log_params(params)
    
    booster = xgb.train(params = params,
                    dtrain = train,
                    num_boost_round = 1000,
                    evals =[(valid,'validation')],
                    early_stopping_rounds = 50)

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric('rmse', rmse)

    with open('model/preprocessor.b', 'wb') as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact('model/preprocessor.b', artifact_path='preprocessor')

    mlflow.xgboost.log_model(booster, artifact_path='model_mlflow')

[11:54:25] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:17.44041
[1]	validation-rmse:14.68804
[2]	validation-rmse:12.58747
[3]	validation-rmse:11.00562
[4]	validation-rmse:9.82595
[5]	validation-rmse:8.96148
[6]	validation-rmse:8.33128
[7]	validation-rmse:7.87864
[8]	validation-rmse:7.55235
[9]	validation-rmse:7.31517
[10]	validation-rmse:7.14547
[11]	validation-rmse:7.02208
[12]	validation-rmse:6.93134
[13]	validation-rmse:6.86199
[14]	validation-rmse:6.81195
[15]	validation-rmse:6.77212
[16]	validation-rmse:6.74095
[17]	validation-rmse:6.71596
[18]	validation-rmse:6.69795
[19]	validation-rmse:6.68378
[20]	validation-rmse:6.67428
[21]	validation-rmse:6.66679
[22]	validation-rmse:6.66122
[23]	validation-rmse:6.65665
[24]	validation-rmse:6.65238
[25]	validation-rmse:6.64856
[26]	validation-rmse:6.64467
[27]	validation-rmse:6.64121
[28]	validation-rmse:6.63796
[29]	validation-rmse:6.63532
[30]	validation-r

In [20]:

logged_model = 'runs:/04548a1800c749b5bd350e800c890562/model_mlflow'

# load model as PyFuncModel

loaded_model = mlflow.pyfunc.load_model(logged_model)

[12:10:27] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


In [21]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: model_mlflow
  flavor: mlflow.xgboost
  run_id: 04548a1800c749b5bd350e800c890562

In [22]:
# type 2

xgboost_model = mlflow.xgboost.load_model(logged_model)

[12:10:27] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


In [23]:
xgboost_model

In [24]:
y_pred = xgboost_model.predict(valid)

In [25]:
y_pred[:10]

array([14.9239235,  7.2020226, 13.471462 , 24.40734  ,  9.389727 ,
       17.185019 , 11.855476 ,  8.34969  ,  8.991192 , 10.490501 ],
      dtype=float32)

In [14]:
from sklearn.ensemble import GradientBoostingRegressor

In [17]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("model/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

2022/10/17 13:42:32 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/Asus/miniconda3/envs/Mlflow/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."
/home/Asus/miniconda3/envs/Mlflow/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
